In [72]:
import bs4
import requests
import time
import pandas as pd
import openpyxl
import csv

In [50]:
def get_page(url, params=None):
    res = requests.get(url, params=params)
    res.raise_for_status()
    return res.text

def get_soup(page):
    return bs4.BeautifulSoup(page, 'html.parser')

# function that extracts the text of the h1 element with class "fx-product-headline product-title__title"
def get_product_name(soup):
    # Auswahl des h1-Elements mit der spezifischen Klasse
    product_name_element = soup.select_one('h1.fx-product-headline.product-title__title')
    
    # Überprüfung, ob das Element gefunden wurde
    if product_name_element:
        # Rückgabe des Textinhalts des Elements
        return product_name_element.get_text().strip()
    else:
        # Rückgabe eines leeren Strings oder einer Fehlermeldung, falls das Element nicht gefunden wird
        return "Produktname nicht gefunden"

# fucntion that extracts the price from the div of class "price"
def get_price(soup):
    price_element = soup.select('div[class="price"]')
    if price_element:
        return price_element[0].get_text().strip().replace(u'\xa0', u' ')
    else:
        return "Preis nicht gefunden"
    return 

In [51]:
# function for getting all product links from a page (objects of class "product__content")
def get_product_links(soup):
    product_links = []
    product_elements = soup.select('a[class="product__content"]')
    
    # extrat the href attribute from each element
    for element in product_elements:
        product_links.append(element.get('href'))
    return product_links

In [67]:
base_url = "https://www.thomann.de/de/"
instrument_urls = {
    "geigensaiten": "alle-produkte-der-kategorie-saiten-fuer-violinen.html",
    "bratschensaiten": "saiten_fuer_violen_-_bratschen.html",
    "cellosaiten": "alle-produkte-der-kategorie-saiten-fuer-celli.html",
}

def create_url(extension, base_url=base_url):
    return base_url + "/" + extension

req_params = {
    'ls': '100',
    'manufacturer[]': [
        'Daddario', 'Jargar', 'Kaplan', 'Pirastro', 'Prim', 
        'Thomastik', 'W.E. Hill & Sons', 'Warchal', 'Westminster', 'Optima'
    ],
    'filter': 'true',
    'setViewMode': 'list'
}

product_df = pd.DataFrame(columns=['name', 'price', 'link'])

for instrument, instrument_url in instrument_urls.items():
    # loop thorugh the instrument_subpages and create a list of all product links
    page_number = 1
    product_links = []
    instrument_df = pd.DataFrame(columns=['name', 'price', 'link'])
    
    while True:
        req_params['pg'] = page_number
        intrument_products_page = get_page(create_url(instrument_url), params=req_params)
        intrument_products_soup = get_soup(intrument_products_page)
        # get all product links from the page
        page_product_links = get_product_links(intrument_products_soup)
        if len(page_product_links) == 0:
            break
        product_links.extend(page_product_links)
        page_number += 1
        time.sleep(1)

    # loop through the product links and extract the product name and price
    for i, product_link in enumerate(product_links):
        product_page = get_page(create_url(product_link))
        product_soup = get_soup(product_page)
        product_name = get_product_name(product_soup)
        product_price = get_price(product_soup)

        instrument_df.loc[len(product_df), 'name'] = product_name
        instrument_df.loc[len(product_df) - 1, 'price'] = product_price
        instrument_df.loc[len(product_df) - 1, 'link'] = create_url(product_link)
        instrument_df.loc[len(product_df) - 1, 'instrument'] = instrument

        time.sleep(0.5)
        instrument_df.to_csv(f'thomann_prices_{instrument}.csv', index=False)

        if i%10 == 0:
            print(f"Produkt {i} von {len(product_links)} in {instrument} abgerufen")

    instrument_df.to_csv(f'thomann_prices_{instrument}.csv', index=False)

    # add instrument_df to prduct_df
    product_df = pd.concat([product_df, instrument_df], ignore_index=True)


Produkt 0 von 405 in geigensaiten abgerufen
Produkt 10 von 405 in geigensaiten abgerufen
Produkt 20 von 405 in geigensaiten abgerufen
Produkt 30 von 405 in geigensaiten abgerufen
Produkt 40 von 405 in geigensaiten abgerufen
Produkt 50 von 405 in geigensaiten abgerufen
Produkt 60 von 405 in geigensaiten abgerufen
Produkt 70 von 405 in geigensaiten abgerufen
Produkt 80 von 405 in geigensaiten abgerufen
Produkt 90 von 405 in geigensaiten abgerufen
Produkt 100 von 405 in geigensaiten abgerufen
Produkt 110 von 405 in geigensaiten abgerufen
Produkt 120 von 405 in geigensaiten abgerufen
Produkt 130 von 405 in geigensaiten abgerufen
Produkt 140 von 405 in geigensaiten abgerufen
Produkt 150 von 405 in geigensaiten abgerufen
Produkt 160 von 405 in geigensaiten abgerufen
Produkt 170 von 405 in geigensaiten abgerufen
Produkt 180 von 405 in geigensaiten abgerufen
Produkt 190 von 405 in geigensaiten abgerufen
Produkt 200 von 405 in geigensaiten abgerufen
Produkt 210 von 405 in geigensaiten abgerufen

In [68]:
product_df

,name,price,link,instrument
0,Thomastik 131 A Dominant Violin-Einzelsaite A,11 €,https://www.thomann.de/de//thomastik_dominant_...,https://www.thomann.de/de//thomastik_dominant_...
1,Thomastik Peter Infeld Violin 4/4 Platin,79 €,https://www.thomann.de/de//thomastik_peter_inf...,https://www.thomann.de/de//thomastik_peter_inf...
2,Pirastro Evah Pirazzi Saitensatz für Violine,88 €,https://www.thomann.de/de//pirastro_evah_piraz...,https://www.thomann.de/de//pirastro_evah_piraz...
3,Thomastik Dominant 135 4/4 Violinsaiten,49 €,https://www.thomann.de/de//thomastik_dominant....,https://www.thomann.de/de//thomastik_dominant....
4,Thomastik Vision Titanium Violinsaiten 4/4,59 €,https://www.thomann.de/de//thomastik_vision_ti...,https://www.thomann.de/de//thomastik_vision_ti...
...,...,...,...,...
737,Pirastro Perpetual Cello 4/4 G strong,109 €,https://www.thomann.de/de//pirastro_perpetual_...,https://www.thomann.de/de//pirastro_perpetual_...
738,Thomastik Dominant D Cello 4/4 light,36 €,https://www.thomann.de/de//thomastik_dominant_...,https://www.thomann.de/de//thomastik_dominant_...
739,Prim Cello String A Soft,"21,90 €",https://www.thomann.de/de//prim_cello_string_a...,https://www.thomann.de/de//prim_cello_string_a...
740,Prim Cello String G Orchestra,"28,90 €",https://www.thomann.de/de//prim_cello_string_g...,https://www.thomann.de/de//prim_cello_string_g...


In [69]:
product_df.to_csv('thomann_prices.csv', index=False)

In [73]:
# product_df to excel
product_df.to_excel('thomann_prices.xlsx', index=False)

In [74]:
import csv